In [1]:
import geopandas as gpd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
import os
import fiona
import pandas as pd
import sys
#%matplotlib inline  # jupyter "magic" to display plots in notebook
pd.set_option('mode.chained_assignment', None)
import numpy as np
from bs4 import BeautifulSoup
import re
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
from geojson import Point, Feature, FeatureCollection, dump

import sys
sys.path.append("../")
import src.soporteScrapping as scr

parameters

In [2]:
url = 'https://download.geofabrik.de/europe/italy/'
country = 'IT'
country_clean = 'IT'

get links

In [3]:
links,flattened = scr.getLinks(url)
links

200


['https://download.geofabrik.de/europe/italy/centro-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/isole-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/nord-ovest-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/sud-latest-free.shp.zip']

In [4]:
flattened

['centro-latest-free.shp.zip',
 'isole-latest-free.shp.zip',
 'nord-est-latest-free.shp.zip',
 'nord-ovest-latest-free.shp.zip',
 'sud-latest-free.shp.zip']

data conversion table input

In [5]:
ruta = os.path.dirname(os.getcwd()) + '\\data'

In [6]:
inPOIS = pd.ExcelFile(f'{ruta}\\POIS_Classification' + '.xlsx')
sheets_list = inPOIS.sheet_names[0]
POIS = pd.read_excel(inPOIS, sheet_name = sheets_list)
POIS.rename({'Tipo2':'Subtipo', 'Subtipo':'fclass'}, axis = 1, inplace=True)
POIS.head()

c:\Users\annas\anaconda3\envs\iron\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Tipo,Subtipo,fclass
0,Accommodation,Hospitality,alpine_hut
1,Accommodation,Hospitality,camp_site
2,Accommodation,Hospitality,chalet
3,Accommodation,Hospitality,guesthouse
4,Accommodation,Hospitality,hostel


downloading shapefile

In [7]:
print('Downloading shapefile...')
patron = r'(.*).shp.zip'
for link, flat in zip(links, flattened):
    local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_{country}'
    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content), mode='a')
    folder = re.findall(patron, flat)
    os.mkdir(os.path.join(local_path, folder[0]))
    z.extractall(path=(local_path + '//'+ folder[0])) # extract to folder
    print(f"Done with {link}")
filenames = [y for y in sorted(z.namelist()) for starting in ['gis_osm_landuse_a_free_1', 'gis_osm_buildings_a_free_1'] if y.startswith(starting)] 
print(filenames)

Done with https://download.geofabrik.de/europe/italy/centro-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/isole-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/nord-ovest-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/sud-latest-free.shp.zip
['gis_osm_buildings_a_free_1.cpg', 'gis_osm_buildings_a_free_1.dbf', 'gis_osm_buildings_a_free_1.prj', 'gis_osm_buildings_a_free_1.shp', 'gis_osm_buildings_a_free_1.shx', 'gis_osm_landuse_a_free_1.cpg', 'gis_osm_landuse_a_free_1.dbf', 'gis_osm_landuse_a_free_1.prj', 'gis_osm_landuse_a_free_1.shp', 'gis_osm_landuse_a_free_1.shx']


POIs

In [8]:
alldf_pois = []
for subdir, dirs, files in os.walk(local_path):
    #global alldf_landuse
    for file in files:
        #print(file)
        if file.startswith('gis_osm_pois_free_1.shp'):
            print(file, subdir)
            df_pois = gpd.read_file(subdir + '\\' + 'gis_osm_pois_free_1' + '.shp')
            alldf_pois.append(df_pois)

gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\centro-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\isole-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\nord-est-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\nord-ovest-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\sud-latest-free


In [9]:
df_pois = pd.concat([(alldf_pois[n]) for n in list(range(len(alldf_pois)))])

In [10]:
print(df_pois.shape)
df_pois.head()

(727713, 5)


,osm_id,code,fclass,name,geometry
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276)
1,26859360,2422,camp_site,Albinia,POINT (10.52806 43.01722)
2,26859373,2422,camp_site,Argentario Camping Village,POINT (11.19245 42.49649)
3,26859377,2422,camp_site,Camping Village Assisi,POINT (12.57420 43.07560)
4,26859385,2422,camp_site,Badiaccia,POINT (12.01639 43.18028)


merge with equivalence table

In [11]:
POIS.head()

,Tipo,Subtipo,fclass
0,Accommodation,Hospitality,alpine_hut
1,Accommodation,Hospitality,camp_site
2,Accommodation,Hospitality,chalet
3,Accommodation,Hospitality,guesthouse
4,Accommodation,Hospitality,hostel


In [12]:
df_pois = df_pois.merge(POIS[['Tipo','Subtipo', 'fclass']], on = 'fclass', how = 'left')
df_pois.value_counts()

TypeError: '<' not supported between instances of 'Point' and 'Point'

In [13]:
df_pois.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 727713 entries, 0 to 727712
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   osm_id    727713 non-null  object  
 1   code      727713 non-null  int64   
 2   fclass    727713 non-null  object  
 3   name      335335 non-null  object  
 4   geometry  727713 non-null  geometry
 5   Tipo      619667 non-null  object  
 6   Subtipo   727709 non-null  object  
dtypes: geometry(1), int64(1), object(5)
memory usage: 44.4+ MB


In [14]:
df_pois.head()

,osm_id,code,fclass,name,geometry,Tipo,Subtipo
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276),Art & Culture,Art & Culture
1,26859360,2422,camp_site,Albinia,POINT (10.52806 43.01722),Accommodation,Hospitality
2,26859373,2422,camp_site,Argentario Camping Village,POINT (11.19245 42.49649),Accommodation,Hospitality
3,26859377,2422,camp_site,Camping Village Assisi,POINT (12.57420 43.07560),Accommodation,Hospitality
4,26859385,2422,camp_site,Badiaccia,POINT (12.01639 43.18028),Accommodation,Hospitality


reproject

In [15]:
df_pois.crs ## tiene que estar en WGS 84

<Geographic 2D CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
df_pois.to_crs("EPSG:4326")

,osm_id,code,fclass,name,geometry,Tipo,Subtipo
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276),Art & Culture,Art & Culture
1,26859360,2422,camp_site,Albinia,POINT (10.52806 43.01722),Accommodation,Hospitality
2,26859373,2422,camp_site,Argentario Camping Village,POINT (11.19245 42.49649),Accommodation,Hospitality
3,26859377,2422,camp_site,Camping Village Assisi,POINT (12.57420 43.07560),Accommodation,Hospitality
4,26859385,2422,camp_site,Badiaccia,POINT (12.01639 43.18028),Accommodation,Hospitality
...,...,...,...,...,...,...,...
727708,10617230772,2082,school,"Terzo circolo didattico - Plesso ""San Rocco""",POINT (14.20147 40.92983),Education,Education
727709,10620295393,2301,restaurant,Les Balais Eventi,POINT (14.44163 40.79405),Restaurant,Restaurant
727710,10620382845,2301,restaurant,Villa Gardenia Ricevimenti,POINT (14.44367 40.78559),Restaurant,Restaurant
727711,10620391065,2301,restaurant,Palazzo Cifelli Eventi,POINT (14.44484 40.78613),Restaurant,Restaurant


In [17]:
df_pois.crs ## tiene que estar en WGS 84

<Geographic 2D CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

export

In [20]:
local_path = os.path.dirname(os.getcwd()) + f'\\data'
local_path

'e:\\Cursos\\00ironhack_DA\\00LABS\\vuelve-conmigo-a-italia\\data'

In [21]:
df_pois.to_file(f'{local_path}\\df_pois.geojson', driver='GeoJSON')  

In [22]:
df_pois.head(1)

,osm_id,code,fclass,name,geometry,Tipo,Subtipo
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276),Art & Culture,Art & Culture


In [25]:
df_pois['lon'] = df_pois.geometry.apply(lambda p: p.x)
df_pois['lat'] = df_pois.geometry.apply(lambda p: p.y)
df_pois.head(1)

,osm_id,code,fclass,name,geometry,Tipo,Subtipo,lon,lat
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276),Art & Culture,Art & Culture,11.636337,42.662761


In [27]:
df_pois.to_csv(f'{local_path}//pois.csv', encoding='utf-8', sep='|', decimal='.', index=False)

In [29]:
local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_IT'
local_path

'e:\\Cursos\\00ironhack_DA\\00LABS\\vuelve-conmigo-a-italia\\data\\tmpOSM_IT'

In [31]:
df_pois_v2 = gpd.read_file(local_path + '\\' + 'pois_IT.gpkg')
df_pois_v2.head(1)

,osm_id,code,fclass,name,Tipo,Subtipo,cod_00,nom_00,cod_01,nom_01,cod_02,nom_02,cod_03,nom_03,cod_04,nom_04,cod_05,nom_05,cod_06,nom_06,cod_07,center_x,center_y,id_cod_00,id_cod_01,id_cod_02,id_cod_03,id_cod_04,id_cod_05,id_cod_06,id_cod_07,Area_ha,Costa,Capital_Re,Capital_Pr,Capital,cod_08,id_cod_08,geometry
0,21027591,2733,archaeological,Via cava del Cavone,Art & Culture,Art & Culture,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano,OMI_I841_R1,Sorano - Zona OMI R1,NaN,NaN,NaN,NaN,530260000006,11.649812,42.672369,57363.0,57372.0,57472.0,1238525.0,1293240.0,NaN,NaN,208742.0,1617.534491,Resto,Resto,Resto,Resto,58010,470379.0,POINT (11.63634 42.66276)


In [32]:
df_pois_v2['nom_01'].unique()

array(['Toscana', 'Umbria', 'Lazio', 'Marche', 'Emilia-Romagna', 'Puglia',
       'Liguria', 'Abruzzo', 'Molise', 'Campania', 'Sardegna', 'Sicilia',
       'Trentino-Alto Adige', 'Veneto', 'Friuli Venezia Giulia',
       'Lombardia', 'Piemonte', "Valle d'Aosta", 'Calabria', 'Basilicata'],
      dtype=object)

In [36]:
df_pois_v2['lon'] = df_pois_v2.geometry.apply(lambda p: p.x)
df_pois_v2['lat'] = df_pois_v2.geometry.apply(lambda p: p.y)

In [39]:
df_pois_v2[df_pois_v2['nom_01'] == 'Sicilia'].to_file(f'{local_path}\\df_pois.geojson', driver='GeoJSON')  